In [1]:
[1,2,3,4,5,6,7,][:5]

[1, 2, 3, 4, 5]

In [10]:
import string

import nltk
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import tokenizer_from_json



MAX_LEN = 64
SEED = 420
# sets random, np.random and tf.random seed
tf.keras.utils.set_random_seed(
    SEED
)

In [16]:
rhyme_model_name = "rhyme_model_23"
tokenizer_path = 'tokenizer_config.json'

with open(tokenizer_path) as f:
    tokenizer_config = f.read()

tokenizer = tokenizer_from_json(tokenizer_config)

rhyme_model = model = load_model(f"models/{rhyme_model_name}.hdf5")

In [14]:
def tokenize_input(phrase, tokenizer):
    tokenized_phrase = tokenizer.texts_to_sequences([phrase])[0]

    if len(tokenized_phrase) < MAX_LEN:
        length_to_pad = MAX_LEN - len(tokenized_phrase)
        phrase_for_output = ([0] * length_to_pad) + tokenized_phrase
    elif len(tokenized_phrase) > MAX_LEN:
        phrase_for_output = tokenized_phrase[-MAX_LEN:]
    else:
        phrase_for_output = tokenized_phrase
    return tf.constant(phrase_for_output, dtype=tf.float64)

In [17]:
df = pd.read_csv("tita_rhymes_poems.tsv", sep="\t")

punctuation = string.punctuation + "«»—"

def remove_punctuation(stanza):
    lines = stanza.split("\n")
    clean_lines = [" ".join([t for t in nltk.tokenize.word_tokenize(line, language='norwegian', preserve_line=True) if t not in punctuation]) for line in lines]
    return clean_lines

df["clean_lines"] = df["stanza"].apply(remove_punctuation)

df["tokenized"] = [tf.convert_to_tensor([tokenize_input(line, tokenizer) for line in clean_lines]) for clean_lines in df["clean_lines"]]

df

,rhyme scheme,stanza,filename,stanza number (in file),clean_lines,tokenized
0,AABCCBI,"Gud, la oss i din kunnskap fremmes,\nså det vi...",2009013000065_1_annotated.txt,0,"[Gud la oss i din kunnskap fremmes, så det vi ...","((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
1,TAABCCB,ANNEN SANG\nFra Landego seiles mot Skrova i no...,2007011001018_11_annotated.txt,0,"[ANNEN SANG, Fra Landego seiles mot Skrova i n...","((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
2,AABCCB,Og er det en høstkveld når mørket står på\nså ...,2007011001018_11_annotated.txt,1,"[Og er det en høstkveld når mørket står på, så...","((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
3,AABCCB,"Snart ligger og kaien der dyngvåt av regn,\nog...",2007011001018_11_annotated.txt,2,"[Snart ligger og kaien der dyngvåt av regn, og...","((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
4,AABCCB,Og rett bort i øst ligger Svinøya gård\nhvor B...,2007011001018_11_annotated.txt,3,"[Og rett bort i øst ligger Svinøya gård, hvor ...","((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
...,...,...,...,...,...,...
5153,AABB,Lukket! Lukket! Øde veier! Gledens vinger brut...,2011041408091_18_annotated.txt,5,"[Lukket Lukket Øde veier Gledens vinger brutt,...","((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
5154,AABB,Og nå stod han her som fremmed i et fremmed la...,2011041408091_18_annotated.txt,6,[Og nå stod han her som fremmed i et fremmed l...,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
5155,AABB,"Opp av havets blanke bølger, som en vårfrisk m...",2011041408091_18_annotated.txt,7,[Opp av havets blanke bølger som en vårfrisk m...,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
5156,AABB,"Til Europa! Tanken slår ham plutselig som lyn,...",2011041408091_18_annotated.txt,8,"[Til Europa Tanken slår ham plutselig som lyn,...","((tf.Tensor(0.0, shape=(), dtype=float64), tf...."


In [18]:
letters = "---"+string.ascii_uppercase.replace("T", "").replace("I","").replace("N","")


def clean_scheme(scheme):
    def scheme_to_numeric(scheme):
        scheme = scheme.replace("T", "0")
        scheme = scheme.replace("I", "1")
        scheme = scheme.replace("N", "2")
        unique_chars = (set(scheme) - {"0", "1", "2"})
        unique_chars = sorted(list(unique_chars), key=lambda x: scheme.index(x))
        scheme = "-".join(scheme)    
        for i, c in enumerate(unique_chars, start=3):
            scheme = scheme.replace(c, str(i))
        return scheme

    def numeric_to_scheme(scheme):
        unique_chars = sorted(list(set(scheme.split("-")) - {"0", "1", "2"}), key=int)
        for i, c in enumerate(unique_chars, start=3):
            scheme = scheme.replace(c, letters[i])
        scheme = scheme.replace("-", "")
        scheme = scheme.replace("0", "T")
        scheme = scheme.replace("1", "I")
        scheme = scheme.replace("2", "N")
        return scheme

    scheme = scheme_to_numeric(scheme)
    scheme = numeric_to_scheme(scheme)
    return scheme

def annotate(model, prev_code, stanza, tokenz):
    code = ["X"]*len(stanza)
    
    for i, line in enumerate(stanza):
        if prev_code[i] in ("I", "T", "N"):
            code[i] = prev_code[i]
            continue
        if  code[i] != "X":
            continue
        letter = letters[i+3]
        code[i] = letter
        
        for j in range(i+1, len(stanza)):
            if prev_code[j] in ("I", "T", "N") or code[j] != "X":
                continue
            
            t1 = tf.reshape(tokenz[i],(1, 64))
            t2 = tf.reshape(tokenz[j], (1, 64))
            rh = model.predict([t1, t2])
            #print(f"Line 1:\n{line}\nLine 2:\n{stanza[j]}\nRhyme:{rh}\n")
            if rh[0][0] > 0.5:
                code[j] = letter
    
    return clean_scheme("".join(code))


In [19]:
annotations = []
for e in df.itertuples():
    annotations.append(annotate(model, e._1, e.clean_lines, e.tokenized))

In [20]:
i = 0
for a1, a2, stanza in zip(df["rhyme scheme"], annotations, df["stanza"]):
    if a1 != a2:
        i += 1
        if len(a1) <= 4:
            print(a1)
            print(a2)
            print(stanza)
            print("")
i, len(df)

ABAB
ABCB
Den første rettergang var mer enn nok.
Sin rases ektefødte sønn er han!
Han er en utsøkt fører for den flokk
hvis skjulte verk er rov og mord og brann.

ABCB
ABCD
Og når du kommer, ta min kappe med;
ti kald er cellens luft og hårdt mitt leie.
Og bøkene hos Karpus, som du vet,
av jordisk gods det er mitt hele eie.

ABCA
ABAA
Han sover.
Et leie er redt til to,
men ingen kommer.
Det er ikke hestehover

ABAB
ABAC
På dem med flit at bygge,
i dem med kunst at bo
som fugl der ligger trygge
i bølgedalens ro

ABAB
ABAC
Fra høydene faller en lysning vakt
på den kulsorte klippevegg.
Der ligger en hummer og fråder svakt
med sin klo i likets skjegg...

ABAB
AAAB
En grav er bedre enn alle slotter,
der har vi hjemme så små vi er.
Guds helgner bodde i hull og grotter,
i graven er vi ham alle nær.

ABAB
ABAC
Så kald på døden når du vil hvile,
han kommer lett som en aftenvind.
Og før det rasler i gravens pile
er du til stjernernes fred gått inn!

AABB
AABC
«Gå, Obadja!l Bring mitt bud
til din 

(1877, 5158)